
# Module 3 — **Agents** 
**Goal:** A compact, teachable reference that mirrors the official docs for Agno Agents and the core subtopics.



## 0) Agents — What & Why (Quick Overview)

**Agents are AI programs where a language model controls the flow of execution.**  
Core loop = **Model** (decides) + **Instructions** (behavior) + **Tools** (actions). They can also use **Memory**, **Storage**, **Knowledge (Agentic RAG)**, and **Reasoning**.

```mermaid
flowchart LR
    subgraph Agent
        I[Instructions] --> M[(Model)]
        M -->|think/plan| R[Reasoning]
        M -->|call| T[Tools]
        M -->|respond| O[Output]
        MEM[(Memory/Session)] --> M
        KN[(Knowledge RAG)] --> M
    end
    ST[(Storage/DB)] --- MEM
    
    style I fill:#e1f5fe
    style M fill:#fff3e0
    style R fill:#f3e5f5
    style T fill:#e8f5e8
    style O fill:#e8f5e8
    style MEM fill:#e3f2fd
    style KN fill:#e3f2fd
    style ST fill:#e3f2fd
```


### Guide Map (where each topic fits)

| Topic | You learn to… |
|---|---|
| **Building Agents** | Assemble model + tools + instructions; minimal agent |
| **Running Agents** | Use `run()`, async/streaming, events, user/session |
| **Debugging** | Enable debug mode; CLI app; inspect flows |
| **Sessions** | Multi‑turn threads, summaries, history, storage flags |
| **Input & Output** | Strings, structured I/O with Pydantic, parser/output models |
| **Context** | Build system/user messages; few-shot; caching |
| **Dependencies** | Inject variables/functions into context dynamically |
| **Agent State** | Persist `session_state`; agentic state tool |
| **Memory** | User memories; agent-managed updates |
| **Knowledge** | Agentic RAG (search at runtime) |
| **Tools** | Toolkits, custom tools, MCP |
| **Multimodal** | Images/audio/video/files in & out |
| **Hooks** | Pre-/Post- validation & transforms |
| **Metrics** | Tokens, timing, per-run/session stats |



## 1) Building Agents (start small → layer features)

Start with **model + tools + instructions** and iterate.


In [ ]:
# Minimal "report" agent (use after setting provider creds)
from agno.agent import Agent
from agno.models.anthropic import Claude
from agno.tools.hackernews import HackerNewsTools

agent = Agent(
    model=Claude(id="claude-sonnet-4-5"),
    tools=[HackerNewsTools()],
    instructions="Write a report on the topic. Output only the report.",
    markdown=True,
)
agent.print_response("Trending startups and products.", stream=True)


## 2) Running Agents (the loop)

**Run loop (concept):** build context → call model → (maybe) tool call → update context → repeat → final message.

```mermaid
sequenceDiagram
  participant You
  participant A as Agent
  participant M as Model
  participant T as Tools
  You->>A: run(input)
  A->>M: context (system, user, history, memories, state, deps)
  M-->>A: message or tool call
  alt tool call
    A->>T: execute(tool, args)
    T-->>A: result
    A->>M: updated context (with tool result)
  end
  M-->>A: final message
  A-->>You: RunOutput / stream
```


In [ ]:
# Blocking vs streaming (reference)
from typing import Iterator
from agno.agent import Agent, RunOutput, RunOutputEvent, RunEvent
from agno.models.anthropic import Claude
from agno.tools.hackernews import HackerNewsTools
from agno.utils.pprint import pprint_run_response

agent = Agent(
    model=Claude(id="claude-sonnet-4-5"),
    tools=[HackerNewsTools()],
    instructions="Write a report on the topic. Output only the report.",
    markdown=True,
)

# Blocking
response: RunOutput = agent.run("Trending startups and products.")
print(response.content)

# Streaming (content chunks)
stream: Iterator[RunOutputEvent] = agent.run("Trending products", stream=True)
for chunk in stream:
    if chunk.event == RunEvent.run_content:
        print(chunk.content)

# Pretty-print stream
stream = agent.run("Trending products", stream=True)
pprint_run_response(stream, markdown=True)


### Streaming & Events (cheatsheet)

Enable `stream=True` to get an iterator of `RunOutputEvent`. Add `stream_events=True` for **all events** (tool calls, reasoning, etc.).

| Category | Event Types (key ones) |
|---|---|
| **Core** | `RunStarted`, `RunContent`, `RunContentCompleted`, `RunCompleted`, `RunError`, `RunCancelled` |
| **Control** | `RunPaused`, `RunContinued` |
| **Tools** | `ToolCallStarted`, `ToolCallCompleted` |
| **Reasoning** | `ReasoningStarted`, `ReasoningStep`, `ReasoningCompleted` |
| **Memory** | `MemoryUpdateStarted`, `MemoryUpdateCompleted` |
| **Session Summary** | `SessionSummaryStarted`, `SessionSummaryCompleted` |
| **Hooks** | `PreHookStarted/Completed`, `PostHookStarted/Completed` |
| **Parser/Output Models** | `ParserModelResponse*`, `OutputModelResponse*` |

You can also define **custom events** by subclassing `CustomEvent` and `yield`ing them from tools.



## 3) Debugging Agents

* Set `debug_mode=True` (agent-wide) or on a specific `run()` call.  
* Use `debug_level=2` for extra detail.  
* Try the built-in **interactive CLI** with `agent.cli_app(stream=True)` during development.


In [ ]:
from agno.agent import Agent
from agno.models.anthropic import Claude
from agno.tools.hackernews import HackerNewsTools

agent = Agent(
    model=Claude(id="claude-sonnet-4-5"),
    tools=[HackerNewsTools()],
    instructions="Write a report on the topic. Output only the report.",
    markdown=True,
    debug_mode=True,
    # debug_level=2,
)

# Terminal debug output
agent.print_response("Trending startups and products.")


## 4) Agent Sessions (multi‑turn + persistence)

**Concepts:** `session_id` (thread), `run_id` (turn), **messages** (model <-> agent protocol).

```mermaid
flowchart LR
    U[User] -->|run| A[Agent]
    A -->|persist| DB[(DB Storage)]
    subgraph DB
        S1[Session: id=S-1]
        S2[Session: id=S-2]
    end
    S1 --- R1[Run r-101]
    S1 --- R2[Run r-102]
    S2 --- R3[Run r-201]
```

---

In [ ]:
# Single session (ids are auto if not provided)
from agno.agent import Agent
from agno.models.openai import OpenAIChat

agent = Agent(model=OpenAIChat(id="gpt-5-mini"))
resp = agent.run("Tell me a 5 second short story about a robot")
print(resp.content, resp.run_id, resp.session_id)

In [ ]:
# Multi-user / multi-session (with SQLite persistence)
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.db.sqlite import SqliteDb

db = SqliteDb(db_file="tmp/data.db")
agent = Agent(model=OpenAIChat(id="gpt-5-mini"), db=db, add_history_to_context=True, num_history_runs=3)

u1, s1 = "user_101", "session_101"
u2, s2 = "user_102", "session_102"

agent.print_response("Tell me a 5 second short story about a robot.", user_id=u1, session_id=s1)
agent.print_response("Now tell me a joke.", user_id=u1, session_id=s1)
agent.print_response("Tell me about quantum physics.", user_id=u2, session_id=s2)
agent.print_response("What is the speed of light?", user_id=u2, session_id=s2)
agent.print_response("Give me a summary of our conversation.", user_id=u1, session_id=s1)


### Session Summaries & History Controls

* Set `enable_session_summaries=True` to auto‑maintain summaries (customize with `SessionSummaryManager`).  
* Add history to context with `add_history_to_context=True` and **optionally** `num_history_runs`.  
* Limit history **tool calls** using `max_tool_calls_from_history=n`.  
* Control storage footprint with: `store_media`, `store_tool_messages`, `store_history_messages`.



## 5) Input & Output (Strings → Structured Data)

Default: **string in → string out**. For reliability, use **Pydantic** schemas.


In [ ]:
# Structured OUTPUT example
from typing import List
from pydantic import BaseModel, Field
from agno.agent import Agent
from agno.models.openai import OpenAIChat

class MovieScript(BaseModel):
    setting: str = Field(..., description="Setting for a blockbuster")
    ending: str = Field(..., description="Provide a happy ending if absent")
    genre: str = Field(..., description="Pick action/thriller/romcom if unknown")
    name: str
    characters: List[str]
    storyline: str

structured_output_agent = Agent(
    model=OpenAIChat(id="gpt-5-mini"),
    description="You write movie scripts.",
    output_schema=MovieScript,
)
structured_output_agent.print_response("New York")

In [ ]:
# Structured INPUT + Validation
from typing import List
from pydantic import BaseModel, Field
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.hackernews import HackerNewsTools

class ResearchTopic(BaseModel):
    topic: str
    focus_areas: List[str] = Field(description="Specific areas to focus on")
    target_audience: str = Field(description="Who this research is for")
    sources_required: int = Field(default=5)

hackernews_agent = Agent(
    name="Hackernews Agent",
    model=OpenAIChat(id="gpt-5-mini"),
    tools=[HackerNewsTools()],
    role="Extract key insights and content from Hackernews posts",
    input_schema=ResearchTopic,  # optional: enforce
)

hackernews_agent.print_response(
    input={"topic": "AI", "focus_areas": ["AI","ML"], "target_audience":"Developers", "sources_required": "5"}
)


**Typesafe agents** = set **both** `input_schema` and `output_schema`.  
You can also improve formatting with a **parser model** or choose a distinct **output model** for the final response.



## 6) Context Engineering (system + user messages)

**System message** = description + instructions + flags (markdown, time, name, location, memories, summary, session state, knowledge filters).

```mermaid
flowchart TB
  D[Description]-->SYS
  INS[Instructions]-->SYS
  FLAGS[Add-ons: time, name, location,<br/>memories, summary, state, filters]-->SYS
  EX[Additional Context / Few-shot]-->SYS
  SYS[System Message]-->CTX[Final Context]
  UM[User Message]-->CTX
  HIST[Chat History]-->CTX
  DEPS[Dependencies]-->UM
```



### System Message Parameters (high-value subset)

| Param | Purpose |
|---|---|
| `description`, `instructions`, `expected_output` | Core behavior / output format |
| `markdown` | Ask model to format in Markdown |
| `add_datetime_to_context`, `add_name_to_context`, `add_location_to_context` | Temporal/identity/location awareness |
| `add_session_summary_to_context`, `add_memories_to_context`, `add_session_state_to_context` | Persisted context |
| `enable_agentic_knowledge_filters` | Let model choose filters for KB search |
| `system_message` | Override entirely |
| `build_context=False` | Disable automatic context building when needed |


In [ ]:
# Directly set/override the system message
from agno.agent import Agent
agent = Agent(system_message="Share a 2 sentence story about love in 12000 CE.")
agent.print_response("Begin!")


**Additional user message context**: set `add_knowledge_to_context` and/or `add_dependencies_to_context=True`.  
**Few-shot**: pass `additional_input=[Message(...), ...]`.  
**Context caching**: keep static stuff at the top of the system message (provider-dependent).



## 7) Dependencies (dynamic injection)

Inject variables/functions into context; reference as `{name}` in instructions or attach the dict to the user message.


In [ ]:
# Static dep + function dep + add to user message
import json, httpx
from agno.agent import Agent
from agno.models.openai import OpenAIChat

def get_user_profile() -> str:
    return json.dumps({"name":"John Doe","experience_level":"senior"}, indent=2)

agent = Agent(
    model=OpenAIChat(id="gpt-5-mini"),
    dependencies={"user_profile": get_user_profile, "plan":"gold"},
    add_dependencies_to_context=True,
    instructions="You are a support agent for plan {plan}.",
    markdown=True,
)
agent.print_response("Summarize my profile and suggest next steps.", stream=True)


## 8) Agent State (session_state)

Maintain per‑session variables (lists, counters, user fields). Update in tools, expose in instructions, and persist via DB.


In [ ]:
# Shopping list state with tools
from textwrap import dedent
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.db.sqlite import SqliteDb

def add_item(session_state, item: str) -> str:
    session_state["shopping_list"].append(item)
    return f"Added '{item}' -> {session_state['shopping_list']}"

def list_items(session_state) -> str:
    return f"Current: {session_state['shopping_list'] or 'empty'}"

agent = Agent(
    model=OpenAIChat(id="gpt-5-mini"),
    session_state={"shopping_list": []},
    db=SqliteDb(db_file="tmp/example.db"),
    tools=[add_item, list_items],
    instructions=dedent("""
      Manage a shopping list. Current: {shopping_list}
    """),
    markdown=True,
)
agent.print_response("Add milk and eggs", stream=True)
print(agent.get_session_state())

In [ ]:
# Agentic session state (auto-manage tool)
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.db.sqlite import SqliteDb

agent = Agent(
    db=SqliteDb(db_file="tmp/agents.db"),
    model=OpenAIChat(id="gpt-5-mini"),
    session_state={"shopping_list": []},
    add_session_state_to_context=True,
    enable_agentic_state=True,
)
agent.print_response("Add bread and apples", stream=True)
print(agent.get_session_state())


## 9) Memory (user personalization)

Let agents **remember user facts**. Either give a tool (`enable_agentic_memory`) or auto‑run a manager (`enable_user_memories`).

In [ ]:
# Memory (Postgres example — adjust creds before running)
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.db.postgres import PostgresDb
from rich.pretty import pprint

user_id = "ava"
db = PostgresDb(db_url="postgresql+psycopg://ai:ai@localhost:5532/ai", memory_table="user_memories")

memory_agent = Agent(
    model=OpenAIChat(id="gpt-4.1"),
    db=db,
    enable_agentic_memory=True,   # or: enable_user_memories=True
    markdown=True,
)

db.clear_memories()

memory_agent.print_response("My name is Ava and I like to ski.", user_id=user_id, stream=True, stream_events=True)
print("Memories about Ava:"); pprint(memory_agent.get_user_memories(user_id=user_id))


## 10) Knowledge (Agentic RAG)

Provide a **Knowledge** base (contents DB + vector DB). Agent searches at runtime (dynamic few‑shot).


In [ ]:
# Agentic RAG sketch (requires Postgres/pgvector setup)
import asyncio
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.knowledge.knowledge import Knowledge
from agno.vectordb.pgvector import PgVector
from agno.db.postgres.postgres import PostgresDb

db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

db = PostgresDb(db_url=db_url, knowledge_table="knowledge_contents")
knowledge = Knowledge(
    contents_db=db,
    vector_db=PgVector(table_name="recipes", db_url=db_url),
)

agent = Agent(model=OpenAIChat(id="gpt-5-mini"), db=db, knowledge=knowledge, markdown=True)

async def setup_and_run():
    await knowledge.add_content_async(
        name="Recipes",
        url="https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf",
        metadata={"user_tag": "Recipes from website"},
    )
    await agent.aprint_response("How do I make chicken and galangal in coconut milk soup?", markdown=True)

# asyncio.run(setup_and_run())


## 11) Tools (toolkits • custom • MCP)

Add pre‑built **toolkits** or simple Python functions. You can also connect **MCP** servers for standard tool access.


In [ ]:
# Toolkit: DuckDuckGo
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(tools=[DuckDuckGoTools()], markdown=True)
agent.print_response("What's happening in France?", stream=True)

In [ ]:
# Custom tool: top HN stories
import json, httpx
from agno.agent import Agent

def get_top_hackernews_stories(num_stories: int = 5) -> str:
    ids = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json').json()
    stories = []
    for sid in ids[:num_stories]:
        s = httpx.get(f'https://hacker-news.firebaseio.com/v0/item/{sid}.json').json()
        s.pop("text", None)
        stories.append(s)
    return json.dumps(stories)

agent = Agent(tools=[get_top_hackernews_stories], markdown=True)
agent.print_response("Summarize the top 5 stories on Hacker News.", stream=True)

In [ ]:
# MCP tools (async sketch)
# from agno.tools.mcp import MCPTools
# mcp_tools = MCPTools(command="uvx mcp-server-git")
# await mcp_tools.connect()
# agent = Agent(tools=[mcp_tools], markdown=True)
# await agent.aprint_response("What is the license for this project?", stream=True)


## 12) Multimodal Agents

Inputs & outputs can be text, images, audio, video, files (provider-dependent).


In [ ]:
# Image input + web search
from agno.agent import Agent
from agno.media import Image
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

img_agent = Agent(model=OpenAIChat(id="gpt-5-mini"), tools=[DuckDuckGoTools()], markdown=True)
img_agent.print_response(
    "Tell me about this image and give me the latest news about it.",
    images=[Image(url="https://upload.wikimedia.org/wikipedia/commons/0/0c/GoldenGateBridge-001.jpg")],
    stream=True,
)


## 13) Pre‑hooks & Post‑hooks

Use hooks to **validate** or **transform** inputs/outputs (guardrails, compliance, sanitization, formatting).


In [ ]:
# Pre-hook example (length check)
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.exceptions import CheckTrigger, InputCheckError
from agno.run.agent import RunInput, AgentSession

def validate_input_length(
    run_input: RunInput,
    session: AgentSession,
    user_id: str | None = None,
    debug_mode: bool | None = None,
) -> None:
    if len(run_input.input_content) > 1000:
        raise InputCheckError("Input too long. Max 1000 chars", check_trigger=CheckTrigger.INPUT_NOT_ALLOWED)

agent = Agent(model=OpenAIChat(id="gpt-5-mini"), pre_hooks=[validate_input_length])

In [ ]:
# Post-hook example (output length)
from agno.exceptions import CheckTrigger, OutputCheckError
from agno.run.agent import RunOutput

def validate_output_length(run_output: RunOutput) -> None:
    if len(run_output.content or "") > 1000:
        raise OutputCheckError("Output too long. Max 1000 chars", check_trigger=CheckTrigger.OUTPUT_NOT_ALLOWED)

agent = Agent(model=OpenAIChat(id="gpt-5-mini"), post_hooks=[validate_output_length])


## 14) Metrics (runs • messages • sessions)

Every `RunOutput` includes **metrics**. Sessions aggregate them.

| Key fields | Meaning |
|---|---|
| `input_tokens`, `output_tokens`, `total_tokens` | Text token accounting |
| `audio_*_tokens` | Audio token accounting |
| `cache_read_tokens`, `cache_write_tokens` | Prompt caching usage |
| `reasoning_tokens` | Reasoning‑token budget used |
| `duration`, `time_to_first_token` | Latency & UX |
| `provider_metrics` | Backend-specific details |


In [ ]:
# Metrics access (sketch)
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.db.sqlite import SqliteDb
from rich.pretty import pprint

agent = Agent(
    model=Gemini(id="gemini-2.5-flash"),
    tools=[DuckDuckGoTools()],
    db=SqliteDb(db_file="tmp/agents.db"),
    markdown=True,
)
run_response = agent.run("What is current news in the world?")

# Per message
if run_response.messages:
    for message in run_response.messages:
        if message.role == "assistant":
            pprint({"metrics": message.metrics.to_dict()})

# Aggregates
pprint({"run_metrics": run_response.metrics.to_dict()})
pprint({"session_metrics": agent.get_session_metrics().to_dict()})


## 15) Quick Checklists

**Before your first run**
- [ ] Provider SDK + API key configured  
- [ ] Minimal agent (model + instructions) works  
- [ ] Add one tool; confirm tool calls

**When adding reliability**
- [ ] `output_schema` defined & validated  
- [ ] `debug_mode=True` during dev  
- [ ] Add pre/post hooks for validation

**When adding persistence & personalization**
- [ ] DB storage set on agent  
- [ ] `user_id`/`session_id` used appropriately  
- [ ] `add_history_to_context` + `num_history_runs` set
